In [10]:
def print_all(list):
    for item in list:
        print(item)

def reverse_tuple(tuples):
    new_tup = tuples[::-1]
    return new_tup

def read_maps(size, n_sp, n_tar, id):
    map_start = []
    map_end = []
    T = []
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_start:
        T_ub = int(file_start.readline())
        for line in file_start.readlines():
            # map(function/type, iterable_object)，把測資用 map 轉整數，然後轉成 list
            map_start += [list(map(int, line.split(',')))]
    
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_end:
        T_ub = int(file_end.readline())
        for line in file_end.readlines():
            map_end += [list(map(int, line.split(',')))]

    T = [i+1 for i in range(T_ub)]
    return T_ub, T, map_start, map_end

def set_start_end(size, map_grid, map_start, map_end, N, R):
    I = []
    P = []
    S_ri = {r:{i:0 for i in N} for r in R}
    F_ri = {r:{i:0 for i in N} for r in R}
    for i in range(size[0]):
        for j in range(size[1]):
            # I
            if map_start[i][j] == 0:
                I.append((0, map_grid[i][j]))
            elif map_start[i][j] != -1:
                I.append((map_start[i][j], map_grid[i][j]))
            # P
            if map_end[i][j] == 0:
                P.append((0, map_grid[i][j]))
            elif map_end[i][j] != -1:
                P.append((map_end[i][j], map_grid[i][j]))
    I = sorted(I)
    P = sorted(P)
    for (r, i) in I:
        S_ri[r][i] = 1
    for (r, i) in P:
        F_ri[r][i] = 1

    return I, P, S_ri, F_ri

def set_arcs(size, map_grid, N, R):
    A = []
    In_ri = {r: {i:[] for i in N} for r in R}
    Out_ri = {r: {i:[] for i in N} for r in R}
    P_ij = {i: {} for i in range(1, size[0]*size[1]+1)}  # 經過格點集合
    Ci = {i: {"Hori":[], "Verti":[]} for i in range(1, size[0]*size[1]+1)}    # 經過節線集合
    Vi = {i:[] for i in N}    # 空格垂直節線進出
    Hi = {i:[] for i in N}    # 空格水平節線進出
    
    for i in range(size[0]):
        for j in range(size[1]):
            node = map_grid[i][j]
            neighbors = [(node,node)]
            spacegrid = []
            space = []
            space_h = [l for l in map_grid[i] if l != node]
            space_v = [l[j] for l in map_grid if l[j] != node]
            space.append(node)
            space.extend(space_h)
            space.extend(space_v)
            
            
            l, r = j-1, j+1
            u, d = i-1, i+1
            temp_l, temp_r, temp_u, temp_d = [], [], [], []
            P_ij[node][node] = []
            while l >= 0:
                temp_l.append(map_grid[i][l])
                P_ij[node][map_grid[i][l]] = temp_l.copy()
                l -= 1
            while r <= size[1] - 1:
                temp_r.append(map_grid[i][r])
                P_ij[node][map_grid[i][r]] = temp_r.copy()
                r += 1
            while u >= 0:
                temp_u.append(map_grid[u][j])
                P_ij[node][map_grid[u][j]] = temp_u.copy()
                u -= 1
            while d <= size[0] - 1:
                temp_d.append(map_grid[d][j])
                P_ij[node][map_grid[d][j]] = temp_d.copy()
                d += 1
            
            
            if j < size[1] - 1:
                right = map_grid[i][j+1]
                neighbors.append((node, right))
            if j > 0:
                left = map_grid[i][j-1]
                neighbors.append((node, left))
            if i < size[0] - 1:
                down = map_grid[i+1][j]
                neighbors.append((node, down))
            if i > 0:
                up = map_grid[i-1][j]
                neighbors.append((node, up))
    
            A.extend(neighbors)
            for h1 in space_h:
                for h2 in space_h:
                    if (h1 < node and h2 > node) or (h1 > node and h2 < node):
                        Ci[node]["Hori"].append((h1,h2))
            for v1 in space_v:
                for v2 in space_v:
                    if (v1 < node and v2 > node) or (v1 > node and v2 < node):
                        Ci[node]["Verti"].append((v1,v2))

            Vi[node] = space_h
            Hi[node] = space_v
            Out_ri[0][node] = [(node,k) for k in space]
            Out_ri[1][node] = neighbors
            In_ri[1] = Out_ri[1].copy()
            
    for r in range(2, R[-1]+1):
        for i in Out_ri[1].keys():
            Out_ri[r][i] = Out_ri[1][i]
        In_ri[r] = Out_ri[r].copy()

    In_ri[0] = Out_ri[0].copy()

    for r in Out_ri:
        for i in Out_ri[r]:
            for arc in Out_ri[r][i]:
                if arc not in A:
                    A.append(arc)
            
    return A, In_ri, Out_ri, P_ij, Ci, Hi, Vi

    
def set_parameter(R, map_s):
    M = 10000000
    epsilon = 0.001
    Num = [0] * len(R)
    for row in map_s:
        for item in row:
            if item != -1: Num[item] += 1
    return M, epsilon, Num

def preprocess(size, n_sp, n_tar, id):
    map_grid = [[i*size[1] + j + 1 for j in range(size[1])] for i in range(size[0])]
    N = [i for i in range(1, size[0]*size[1]+1)]
    R = [i for i in range(n_tar+1)]   # 其中 0 代表空格
    
    T_ub, T, map_s, map_e = read_maps(size, n_sp, n_tar, id)
    A, In_ri, Out_ri, P_ij, Ci, Hi, Vi = set_arcs(size, map_grid, N, R)
    I, P, S_ri, F_ri = set_start_end(size, map_grid, map_s, map_e, N, R)
    M, epsilon, Num = set_parameter(R, map_s)

    return map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, P_ij, Ci, Hi, Vi, I, P, S_ri, F_ri, M, epsilon, Num

# [map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, P_ij, Ci, Hi, Vi, I, P, S_ri, F_ri, M, epsilon, Num] = preprocess((3,3),2,4,4)
# print_all([map_grid, map_s, map_e, T_ub, T, N, R, A, In_ri, Out_ri, P_ij, Ci, Hi, Vi, I, P, S_ri, F_ri, M, epsilon, Num])
# for k, v in P_ij.items():
#     print(k, v)